# Monte-Carlo Tree Search (MCTS)

Note:  a large chunk of this code was AI generated, and I claim no authenticity. My aim is to use this notebook as a playground to understand MCTS in practicality. 

In [2]:
import math
import random

class Node:

    '''

        Declaration of a tree node. Each node has some internal state, a list of children and a parent. 
        We also keep track of the number of visits made to each node and number of wins after having 
        visited said node. 
    
    '''
    
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.wins = 0
        self.visits = 0

    def select_child(self, c=1.4):
        """
        Select a child node using the Upper Confidence Bound for Trees formula.
        """
        return max(self.children, key=lambda child: child.wins / child.visits + c * math.sqrt(2 * math.log(self.visits) / child.visits))

    def expand(self):
        """
        Expand the current node by adding all possible child nodes.
        """
        for move in self.state.available_moves():
            next_state = self.state.make_move(move)
            self.children.append(Node(next_state, parent=self))

    def backpropagate(self, result):
        """
        Update the current node and propagate back to the root.
        """
        self.visits += 1
        self.wins += result
        if self.parent:
            self.parent.backpropagate(1 - result)


class TicTacToe:
    def __init__(self):
        self.board = [' '] * 9
        self.current_player = 'X'

    def available_moves(self):
        return [i for i, spot in enumerate(self.board) if spot == ' ']

    def make_move(self, move):
        new_board = self.board.copy()
        new_board[move] = self.current_player
        next_player = 'O' if self.current_player == 'X' else 'X'
        new_state = TicTacToe()
        new_state.board = new_board
        new_state.current_player = next_player
        return new_state

    def is_terminal(self):
        winning_positions = [(0, 1, 2), (3, 4, 5), (6, 7, 8),
                             (0, 3, 6), (1, 4, 7), (2, 5, 8),
                             (0, 4, 8), (2, 4, 6)]
        for a, b, c in winning_positions:
            if self.board[a] == self.board[b] == self.board[c] != ' ':
                return True
        if ' ' not in self.board:
            return True
        return False

    def get_winner(self):
        # 1 for 'X', 0 for 'O', 0.5 for draw
        if not self.is_terminal():
            return 0.5
        if self.current_player == 'X':
            return 0
        return 1


def mcts(root_state, num_iterations):
    root = Node(root_state)

    for _ in range(num_iterations):
        node = root
        state = root_state

        # Selection
        while node.children and not state.is_terminal():
            node = node.select_child()
            state = node.state

        # Expansion
        if not state.is_terminal():
            node.expand()
            if node.children:
                node = random.choice(node.children)
                state = node.state

        # Simulation
        while not state.is_terminal():
            move = random.choice(state.available_moves())
            state = state.make_move(move)

        # Backpropagation
        winner = state.get_winner()
        node.backpropagate(winner)

    return max(root.children, key=lambda child: child.visits).state.board


In [3]:
root_state = TicTacToe()
result_board = mcts(root_state, 1000)
print(result_board)

ZeroDivisionError: division by zero